In [1]:
# --- CÉLULA 1: Setup e Carregamento de Bibliotecas (Revisada) ---
import pandas as pd
import numpy as np
import joblib 
import os 
from math import sqrt 

# Bibliotecas de Pré-processamento e Métricas
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score

# Modelos para Classificação
from sklearn.naive_bayes import GaussianNB 
from sklearn.linear_model import LogisticRegression 

print("Bibliotecas importadas e prontas para Classificação Pima Indians Diabetes.")

Bibliotecas importadas e prontas para Classificação Pima Indians Diabetes.


In [2]:
# --- CÉLULA 2: Carregamento e Pré-processamento (Pima Indians Diabetes) ---

# URL para o dataset (sem cabeçalho)
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.csv"

# Nomes das colunas conforme a documentação
col_names = [
    'Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 
    'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'
]

# 1. Carregamento
try:
    pima_df = pd.read_csv(url, names=col_names)
    print("Dataset Pima Indians Diabetes carregado com sucesso.")
except Exception as e:
    print(f"Erro ao carregar dataset. Verifique a URL ou a conexão: {e}")

# 2. Definição de X e Y
X = pima_df.drop('Outcome', axis=1) # Todas as features exceto o alvo
y = pima_df['Outcome']               # Alvo: 0 (Não Diabetes), 1 (Diabetes)

# 3. Split
# Usamos stratify=y para garantir que a proporção das classes seja mantida no treino/teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# 4. Padronização (Crucial para Modelos Probabilísticos e Logísticos)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 5. Salvamento dos Dados (Reutilizando a estrutura de diretórios)
DATA_DIR = 'data'
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR) 

joblib.dump(X_train_scaled, os.path.join(DATA_DIR, 'X_train_pima_scaled.joblib'))
joblib.dump(X_test_scaled, os.path.join(DATA_DIR, 'X_test_pima_scaled.joblib'))
joblib.dump(y_train, os.path.join(DATA_DIR, 'y_train_pima.joblib'))
joblib.dump(y_test, os.path.join(DATA_DIR, 'y_test_pima.joblib'))

print(f"Dados Pima Indians Diabetes padronizados e salvos em: {DATA_DIR}/")
print(f"Shape do X_train: {X_train_scaled.shape}")

Dataset Pima Indians Diabetes carregado com sucesso.
Dados Pima Indians Diabetes padronizados e salvos em: data/
Shape do X_train: (537, 8)


In [3]:
# --- CÉLULA 3: Naive Bayes (GaussianNB) ---
from sklearn.naive_bayes import GaussianNB

# 1. Instanciar e Treinar
nb_model = GaussianNB()
# Usamos os dados padronizados (scaled), embora o Naive Bayes não seja sensível à escala, é boa prática.
nb_model.fit(X_train_scaled, y_train)

# 2. Salvar o modelo
MODEL_DIR = 'models'
MODEL_FILENAME = 'naive_bayes_classifier_model.joblib'
MODEL_PATH_NB = os.path.join(MODEL_DIR, MODEL_FILENAME)

if not os.path.exists(MODEL_DIR): # Já foi criado na Célula 3 do notebook anterior, mas é bom garantir
    os.makedirs(MODEL_DIR) 

joblib.dump(nb_model, MODEL_PATH_NB) 
print(f"Modelo Naive Bayes treinado e salvo em: {MODEL_PATH_NB}")

# 3. Avaliação do Modelo

# Previsão da classe (0 ou 1)
y_pred_nb = nb_model.predict(X_test_scaled)
# Previsão da probabilidade da CLASSE 1 (necessário para a métrica AUC-ROC)
y_pred_proba_nb = nb_model.predict_proba(X_test_scaled)[:, 1] 

# Métricas de avaliação
accuracy_nb = accuracy_score(y_test, y_pred_nb)
roc_auc_nb = roc_auc_score(y_test, y_pred_proba_nb)

print(f"\nNaive Bayes: Acurácia (Teste): {accuracy_nb:.4f}")
print(f"Naive Bayes: AUC-ROC (Teste): {roc_auc_nb:.4f}")
print("\nRelatório de Classificação (Naive Bayes):")
print(classification_report(y_test, y_pred_nb))

Modelo Naive Bayes treinado e salvo em: models\naive_bayes_classifier_model.joblib

Naive Bayes: Acurácia (Teste): 0.7446
Naive Bayes: AUC-ROC (Teste): 0.8088

Relatório de Classificação (Naive Bayes):
              precision    recall  f1-score   support

           0       0.80      0.81      0.81       150
           1       0.64      0.62      0.63        81

    accuracy                           0.74       231
   macro avg       0.72      0.72      0.72       231
weighted avg       0.74      0.74      0.74       231



In [4]:
# --- CÉLULA 4: Regressão Logística ---
from sklearn.linear_model import LogisticRegression

# 1. Definir o range do hiperparâmetro C (Inverso da força de regularização)
# Usaremos uma pequena gama de valores para C (regularização L2 padrão)
param_grid_lr = {'C': [0.01, 0.1, 1.0, 10.0]} 

# 2. Configurar o GridSearchCV
# O solver 'liblinear' é robusto para small-to-medium datasets e lida bem com L1/L2
lr = LogisticRegression(solver='liblinear', random_state=42)
lr_grid = GridSearchCV(
    estimator=lr,
    param_grid=param_grid_lr,
    scoring='roc_auc', # Focamos na área sob a curva (AUC-ROC) como métrica de otimização
    cv=5, 
    verbose=0,
    n_jobs=-1
)

# 3. Treinar
# A Regressão Logística é sensível à escala, por isso usamos os dados scaled
lr_grid.fit(X_train_scaled, y_train)

# 4. Salvar o melhor estimador
best_lr_model = lr_grid.best_estimator_

MODEL_DIR = 'models'
MODEL_FILENAME = 'logistic_regression_classifier_model.joblib'
MODEL_PATH_LR = os.path.join(MODEL_DIR, MODEL_FILENAME)

joblib.dump(best_lr_model, MODEL_PATH_LR) 
print(f"Modelo Regressão Logística treinado e salvo em: {MODEL_PATH_LR}")

# 5. Avaliação do Modelo Otimizado
y_pred_lr = best_lr_model.predict(X_test_scaled)
# Previsão da probabilidade da CLASSE 1 (coluna de índice 1)
y_pred_proba_lr = best_lr_model.predict_proba(X_test_scaled)[:, 1] 

# Métricas de avaliação
accuracy_lr = accuracy_score(y_test, y_pred_lr)
roc_auc_lr = roc_auc_score(y_test, y_pred_proba_lr)
best_c = lr_grid.best_params_['C']

print(f"\nRegressão Logística: Melhor C encontrado: {best_c}")
print(f"Regressão Logística: Acurácia (Teste): {accuracy_lr:.4f}")
print(f"Regressão Logística: AUC-ROC (Teste): {roc_auc_lr:.4f}")
print("\nRelatório de Classificação (Regressão Logística):")
print(classification_report(y_test, y_pred_lr))

Modelo Regressão Logística treinado e salvo em: models\logistic_regression_classifier_model.joblib

Regressão Logística: Melhor C encontrado: 1.0
Regressão Logística: Acurácia (Teste): 0.7446
Regressão Logística: AUC-ROC (Teste): 0.8380

Relatório de Classificação (Regressão Logística):
              precision    recall  f1-score   support

           0       0.77      0.87      0.82       150
           1       0.68      0.52      0.59        81

    accuracy                           0.74       231
   macro avg       0.72      0.69      0.70       231
weighted avg       0.74      0.74      0.74       231

